## New Models

In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import absolute
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from skelm import ELMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import time

In [2]:
data=pd.read_csv("Group_2_data_cleaned.csv")
data=pd.DataFrame(data)
X=np.array(data.iloc[0:,3:12])
Y=np.array(data.iloc[0:,12:])

In [3]:
LE=LabelEncoder()
X[:,0]=LE.fit_transform(X[:,0]) #Select labels from variables
#Split train and test data. Test data size is 50%, training set is 50%
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state = 0) 
#The data set is normalized or 'standardized' for a better performaing KNN for both train and test data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



### Non-Linear SVM Classifier

In [4]:
# Set up the pipeline
svm_pipe = make_pipeline(StandardScaler(), SVC())

# Set up the grid search
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['poly', 'rbf', 'sigmoid'],
    'svc__gamma': ['scale', 'auto']
}
clf = GridSearchCV(svm_pipe, param_grid, cv=5)

# Fit the model and time it
clf.fit(X_train, y_train)



C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [0.1, 1, 10],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['poly', 'rbf', 'sigmoid']})

In [5]:
# Print the best parameters and the time taken to fit the model
print('Best parameters:', clf.best_params_)

Best parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}


In [6]:
# Use the best parameters to fit a new SVM model
best_params = clf.best_params_
svm_pipe = make_pipeline(StandardScaler(), SVC(C=best_params['svc__C'], kernel=best_params['svc__kernel'], gamma=best_params['svc__gamma']))
start_time = time.time()
svm_pipe.fit(X_train, y_train)
end_time = time.time()
print(f"Model Fitting Time: {end_time - start_time}")

Model Fitting Time: 0.041069746017456055


C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
y_pred = svm_pipe.predict(X_test)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
print('Accuracy:', round(accuracy_score(y_pred,y_test)*100,2),'%')

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2838
           1       0.95      0.98      0.96       323

    accuracy                           0.99      3161
   macro avg       0.97      0.99      0.98      3161
weighted avg       0.99      0.99      0.99      3161

Confusion Matrix:
 [[2822    7]
 [  16  316]] 

Accuracy: 99.27 %


### Extreme Learning Machine (ELM)

In [8]:
class ELM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ELM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h = F.relu(self.linear1(x))
        y_pred = torch.sigmoid(self.linear2(h))
        return y_pred

In [9]:
# Perform model structure selection
parameters = {'hidden_size': [64, 128], 'batch_size': [32, 64], 'learning_rate': [0.001, 0.01]}

In [10]:
start_time = time.time()
for hidden_size in parameters['hidden_size']:
    for batch_size in parameters['batch_size']:
        for learning_rate in parameters['learning_rate']:
            elm = ELM(X_train.shape[1], hidden_size, 1)
            optimizer = torch.optim.Adam(elm.parameters(), lr=learning_rate)
            criterion = nn.BCELoss()

            # Train the ELM
            for i in range(0, len(X_train), batch_size):
                x = torch.Tensor(X_train[i:i+batch_size])
                y = torch.Tensor(y_train[i:i+batch_size])
                y_pred = elm(x)
                loss = criterion(y_pred, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Test the ELM
            y_pred = elm(torch.Tensor(X_test)).detach().numpy()
            y_pred = [1 if p > 0.5 else 0 for p in y_pred]
end_time = time.time()
print('Time taken:', end_time - start_time)

Time taken: 0.5901801586151123


In [11]:
print('hidden_size: {}, batch_size: {}, learning_rate: {}'.format(hidden_size, batch_size, learning_rate))
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
print('Accuracy:', round(accuracy_score(y_pred,y_test)*100,2),'%')

hidden_size: 128, batch_size: 64, learning_rate: 0.01
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2838
           1       0.85      0.96      0.90       323

    accuracy                           0.98      3161
   macro avg       0.92      0.97      0.94      3161
weighted avg       0.98      0.98      0.98      3161

Confusion Matrix:
 [[2782   14]
 [  56  309]] 

Accuracy: 97.79 %


### Deep Learning 

In [12]:
#Create and initialize deep learning classifier
gaussDL = GaussianNB()
DLmodel = gaussDL.fit(X_train, y_train)

y_pred3 = gaussDL.predict(X_test)


C:\Users\espeh\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_pred3))
print("Accuracy:",round(accuracy_score(y_test, y_pred3),4)*100,'%')


              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2838
           1       0.85      0.96      0.90       323

    accuracy                           0.98      3161
   macro avg       0.92      0.97      0.94      3161
weighted avg       0.98      0.98      0.98      3161

Confusion Matrix:
 [[2758   80]
 [   3  320]]
Accuracy: 97.37 %
